In [2]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential,Model,initializers,layers,Input
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt

Residual_BottleNeck

1. MobileNetV2:  Inverted Residuals  and Linear  Bottlenecks, Figure 4(d) 기반으로 작성

2. Full-pre-activation

3. Hyperparameters: 
 - expansion factor=t
 - output channels=c
 - input channels=k

In [3]:
def Residual_BottleNeck_st_1_Block(x,t,c,k):
    x1=layers.BatchNormalization()(x)
    x1=layers.ReLU(max_value=6)(x1)
    x1=layers.Conv2D(filters=k*t,kernel_size=(1,1),strides=1,padding='same')(x1)
    x1=layers.BatchNormalization()(x)
    x1=layers.ReLU(max_value=6)(x)
    x1=layers.DepthwiseConv2D(kernel_size=(3,3),strides=1,padding='same')(x1)
    x1=layers.Conv2D(filters=c,kernel_size=(1,1),strides=1,padding='same')(x1)

    x2= layers.Add()([x,x1])
    return x2


def Residual_BottleNeck_st_2_Block(x,t,c,k):
    x=layers.BatchNormalization()(x)
    x=layers.ReLU(max_value=6)(x)
    x=layers.Conv2D(filters=k*t,kernel_size=(1,1),strides=1,padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU(max_value=6)(x)
    x=layers.DepthwiseConv2D(kernel_size=(3,3),strides=2,padding='same')(x)
    x=layers.Conv2D(filters=c,kernel_size=(1,1),strides=1,padding='same')(x)

    return x


MobileNet V2

In [8]:
input=Input(shape=(224,224,3))
x=layers.Conv2D(filters=32,kernel_size=(3,3),strides=(2,2),padding='same')(input)

#x=Residual_BottleNeck_st_1_Block(x,1,16)-> add불가능
x=layers.BatchNormalization()(x)
x=layers.ReLU(max_value=6)(x)
x=layers.Conv2D(filters=32,kernel_size=(1,1),strides=1,padding='same')(x)
x=layers.BatchNormalization()(x)
x=layers.ReLU(max_value=6)(x)
x=layers.DepthwiseConv2D(kernel_size=(3,3),strides=1,padding='same')(x)
x=layers.Conv2D(filters=16,kernel_size=(1,1),strides=1,padding='same')(x)

x=Residual_BottleNeck_st_2_Block(x,6,24,16)
x=Residual_BottleNeck_st_1_Block(x,6,24,24)

x=Residual_BottleNeck_st_2_Block(x,6,32,24)
x=Residual_BottleNeck_st_1_Block(x,6,32,32)
x=Residual_BottleNeck_st_1_Block(x,6,32,32)

x=Residual_BottleNeck_st_2_Block(x,6,64,32)
x=Residual_BottleNeck_st_1_Block(x,6,64,64)
x=Residual_BottleNeck_st_1_Block(x,6,64,64)
x=Residual_BottleNeck_st_1_Block(x,6,64,64)

#x=Residual_BottleNeck_st_1_Block(x,6,96,64)-> add불가능
x=layers.BatchNormalization()(x)
x=layers.ReLU(max_value=6)(x)
x=layers.Conv2D(filters=64*6,kernel_size=(1,1),strides=1,padding='same')(x)
x=layers.BatchNormalization()(x)
x=layers.ReLU(max_value=6)(x)
x=layers.DepthwiseConv2D(kernel_size=(3,3),strides=1,padding='same')(x)
x=layers.Conv2D(filters=96,kernel_size=(1,1),strides=1,padding='same')(x)

x=Residual_BottleNeck_st_1_Block(x,6,96,96)
x=Residual_BottleNeck_st_1_Block(x,6,96,96)

x=Residual_BottleNeck_st_2_Block(x,6,160,96)
x=Residual_BottleNeck_st_1_Block(x,6,160,160)
x=Residual_BottleNeck_st_1_Block(x,6,160,160)

#x=Residual_BottleNeck_st_1_Block(x,1,320)->Add 불가능
x=layers.BatchNormalization()(x)
x=layers.ReLU(max_value=6)(x)
x=layers.Conv2D(filters=160*6,kernel_size=(1,1),strides=1,padding='same')(x)
x=layers.BatchNormalization()(x)
x=layers.ReLU(max_value=6)(x)
x=layers.DepthwiseConv2D(kernel_size=(3,3),strides=1,padding='same')(x)
x=layers.Conv2D(filters=320,kernel_size=(1,1),strides=1,padding='same')(x)

x=layers.Conv2D(filters=1280,kernel_size=(1,1),strides=1,padding='same')(x)
x=layers.AveragePooling2D(pool_size=(7,7),strides=1)(x)
x=layers.Conv2D(filters=1280,kernel_size=(1,1),strides=1,padding='same')(x)

x=layers.GlobalAveragePooling2D()(x)
x=layers.Dense(units=1000,activation='softmax')(x)

model = tf.keras.models.Model(input,x)

In [16]:
plot_model(model,to_file='MobileNet_V2.png',show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [9]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_74 (Conv2D)             (None, 112, 112, 32  896         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_68 (BatchN  (None, 112, 112, 32  128        ['conv2d_74[0][0]']              
 ormalization)                  )                                                           

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy' ,metrics=['acc'])

In [ ]:
with tf.device("/device:GPU:0"):
    history=model.fit(train_ds,validation_data=validation_ds,epochs=100,batch_size=16,verbose=1)